In [57]:
from typing import TypedDict, List, Tuple
from langgraph.graph import StateGraph, END, START
from random import randint

In [95]:
class AgentState(TypedDict):
    player : str
    correct : int
    curr : int
    guesses : List[int]
    lower_bound : int
    upper_bound : int
    attempts : int
    max_attempts : int
    message : str

In [184]:
def guess(state:AgentState) -> AgentState:
    ''' this will guess the number'''
    guess = randint(state['lower_bound'],state['upper_bound'])
    while guess in state['guesses']:
        guess = randint(state['lower_bound'],state['upper_bound'])
    # print(guess) #debug
    state['curr'] = guess
    state['attempts'] += 1
    return state


In [185]:
def good_message(state:AgentState) -> AgentState:
    state['message'] = f"Good Job {state['player']}"
    return state


def bad_message(state:AgentState) -> AgentState:
    state['message'] = f"{state['player']}, you missed this round"
    return state

In [ ]:
def validate(state: AgentState) -> str:
    if state['attempts'] <= state['max_attempts']:
        state['guesses'].append(state['curr'])
        if state['curr'] == state['correct']:
            return 'good'
        else:
            return 'bad'
    else:
        return 'end_loop'


In [187]:
graph = StateGraph(AgentState)
graph.add_node('validator', lambda state: state)
graph.add_node('generator', guess)
graph.add_node('good', good_message)
graph.add_node('bad', bad_message)

graph.add_conditional_edges(
    'validator',
    validate , 
    {
        'good' : 'good',
        'end_loop' : END,
        'bad' : "bad"
    }
)

graph.add_edge('generator','validator')
graph.add_edge('good', END)
graph.add_edge('bad', 'generator')
graph.set_entry_point('generator')

app = graph.compile()

In [ ]:
initial_state = {
    'player' : "Henry",
    'correct':      7,
    'curr':        -1,
    'guesses':     [],
    'lower_bound':  0,
    'upper_bound': 10,
    'attempts':     0,
    'max_attempts': 5,
    'message': ''
}

results = app.invoke(initial_state)
print(results['message'])

Good Job Henry [2, 1, 10, 3, 7]
